In [ ]:
from langchain_groq import ChatGroq

In [ ]:
llm = ChatGroq(
    temperature=0,
    groq_api_key="YOUR_GROQ_API_KEY",
    model_name="llama-3.1-8b-instant"
)

In [ ]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://career.fpt-software.com/jobs/machine-learning-software-engineer/")
page_data = loader.load().pop().page_content    
page_data[:500]  

'\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nMachine Learning Software Engineer\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n \n\n\n\n\n\n\n\n\n\n\n \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nJoin Us\n\n\n\n×\nSign Up\n.NET/C# Developer2D/3DADAS/ADAdminAEMAgileAIAkanoxAMSAnalystAndroidAngular DeveloperAngular JSAngular/Angular JSAppianArchitectAutomation TesterAutomotiveAutosarAWSAzureB2BBack officeBack-endBIBlockchainBridge EngineerBrSEBusiness AnalystBusiness IntelligenceBusiness UnitBWCC/ C++CADCAECEOCloudCloud EngineerCloudOpsCMSCobolComt'

In [23]:

from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)

str

In [24]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

[{'role': 'Machine Learning Software Engineer',
  'experience': '5-7 years of EXP',
  'skills': ['Python',
   'Kubeflow',
   'Docker',
   'Clean Code',
   'Version Control',
   'Unit Testing'],
  'description': 'Wrap, deploy, and manage pre-built ML models using Python and Kubeflow. Build data preprocessing and postprocessing pipelines to support ML inference. Collaborate with Data Scientists, ML Ops, and Platform Engineers to ensure smooth integration and scalable solutions.'}]

In [25]:
type(json_res)

list

In [12]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [26]:
import chromadb
import uuid

client = chromadb.PersistentClient("vectorstore")
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [27]:
links = collection.query(query_texts=["Experience in Python","Experience in React Native"], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/react-native-portfolio'},
  {'links': 'https://example.com/react-portfolio'}]]

In [28]:
job = json_res
job

[{'role': 'Machine Learning Software Engineer',
  'experience': '5-7 years of EXP',
  'skills': ['Python',
   'Kubeflow',
   'Docker',
   'Clean Code',
   'Version Control',
   'Unit Testing'],
  'description': 'Wrap, deploy, and manage pre-built ML models using Python and Kubeflow. Build data preprocessing and postprocessing pipelines to support ML inference. Collaborate with Data Scientists, ML Ops, and Platform Engineers to ensure smooth integration and scalable solutions.'}]

In [29]:

prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Mohan, a business development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Mohan, BDE at AtliQ. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Unlock the Full Potential of Your Machine Learning Models with AtliQ

Dear [Client Name],

I came across your job posting for a Machine Learning Software Engineer and was impressed by the requirements. AtliQ, an AI & Software Consulting company, specializes in facilitating seamless integration of business processes through automated tools. Our expertise in machine learning, software development, and process optimization makes us an ideal partner to fulfill your needs.

Our team of experts has extensive experience in wrapping, deploying, and managing pre-built ML models using Python and Kubeflow. We have successfully built data preprocessing and postprocessing pipelines to support ML inference, ensuring smooth integration and scalable solutions. Our expertise in Clean Code, Version Control, and Unit Testing guarantees high-quality deliverables.

AtliQ has a proven track record of empowering enterprises with tailored solutions, fostering scalability, process optimization, cost r